In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import numpy as np

In [ ]:
# Load data of closing transaction
df = pd.read_csv('../csv/df_TS_week.csv')
df.head()

In [ ]:
# Create a new df of the item_id equal to IT01 
df = df[df['item_id'] == 'IT01']

In [ ]:
# Remove idem_id column and target column
df = df.drop(columns=['item_id', 'target'])
df.head()

In [ ]:
# Trasforma le colonne ACT in una singola colonna item_id e i valori corrispondenti in una colonna target
df = df.melt(id_vars=["timestamp"], var_name="item_id", value_name="target")

# Ordina il DataFrame per il timestamp
df.sort_values(by="timestamp", inplace=True)

# Visualizza il DataFrame trasformato
df.head(10)

In [ ]:
# Drop row where item_id is ACT17 and ACT44
df = df[df['item_id'] != 'ACT04']
df = df[df['item_id'] != 'ACT17']
df = df[df['item_id'] != 'ACT44']

In [ ]:
df.head(9)

In [ ]:
prediction_length = 12  # Length of the prediction

# Convert the 'timestamp' column to datetime format if it's not already in that format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Find the maximum date in the dataset
max_date = df['timestamp'].max()

# Calculate the start date of the test set
test_start_date = max_date - pd.DateOffset(weeks=prediction_length+1)

# Split the dataset into training and test sets
train_data = df[df['timestamp'] <= test_start_date]
test_data = df[df['timestamp'] > test_start_date]

# Print the percentage dimensions of the training and test sets
print(train_data.shape[0] / df.shape[0])
print(test_data.shape[0] / df.shape[0])


In [ ]:
# Convert train_data to TimeSeriesDataFrame
train_data = TimeSeriesDataFrame(
    train_data, 
    id_column="item_id",
    timestamp_column="timestamp"
    
)

train_data.head()

In [ ]:
# Convert test_data to TimeSeriesDataFrame
test_data = TimeSeriesDataFrame(
    test_data, 
    id_column="item_id",
    timestamp_column="timestamp"
    
)

test_data.head()

In [ ]:
# Build predictor
predictor = TimeSeriesPredictor(
    prediction_length=12,
    freq='W-MON',
    target='target',
    path='TFT-IT_ACT',
    eval_metric='MASE'
)

In [ ]:
from autogluon.common import space

predictor.fit(
    train_data,
    hyperparameters = {
        "TemporalFusionTransformer": { 
           "hidden_dim": space.Categorical(32, 64, 128),  
           "variable_dim": space.Categorical(32, 64, 128),
           "num_heads": space.Categorical(1,2,4),
           "dropout_rate": space.Real(0.1, 0.3),
           "max_epochs": space.Categorical(50, 100, 150),
           "batch_size": space.Categorical(64, 128, 156),
           "lr": space.Categorical(0.1, 0.01, 0.001),
           "early_stopping_patience" : 20
        },
    },
    hyperparameter_tune_kwargs="auto",
    refit_full=True,
    num_val_windows=3,
    random_seed=42,
    enable_ensemble = False
)

In [ ]:
# Make prediction on training data
predictions = predictor.predict(train_data, random_seed=42)
predictions.head()

In [ ]:
predictor.leaderboard(test_data)

In [ ]:
plt.figure(figsize=(20, 5))
plt.title("Comparison between values predicted by the model and actual values from test data", pad=20, fontsize=15)

item_id = "ACT34"
y_past = train_data.loc[item_id]["target"]
y_pred = predictions.loc[item_id]
y_test = test_data.loc[item_id]["target"][:12]

# Get the dates for each observation
all_dates = list(y_past.index[-8:]) + list(y_test.index)
all_ticks = range(len(all_dates))

# Set the x-axis range for past and future
past_ticks = range(8)
future_ticks = range(8, len(all_dates))

plt.plot(past_ticks, y_past[-8:], label="Past time series values", marker='o', linestyle='-')
plt.plot(future_ticks, y_pred["mean"], label="Mean forecast", marker='o', linestyle='-')
plt.plot(future_ticks, y_test, label="Future time series values", marker='o', linestyle='-')


# Set x-axis labels with dates without time
date_labels = [date.strftime("%Y-%m-%d") for date in all_dates]
plt.xticks(all_ticks, date_labels, rotation=45)

# Add label for x-axis and y-axis
plt.xlabel("Date", fontsize=15, labelpad=15)
plt.ylabel("Number of transactions", fontsize=15, labelpad=15)

plt.legend()
plt.show()